 ### please re-start kernel if these packages are installed for the first time

In [ ]:
# !pip install -e /projects/rea3/AP-ReA/jupyter-ML/pkgs/objFuncs
# !pip install -e /projects/rea3/AP-ReA/jupyter-ML/pkgs/pyBO

### This template is for
- objFuncs: v1.0.4  
- pyBO: v1.0.2

In [ ]:
import objFuncs
from objFuncs import objFuncGoals, construct_machineIO
from objFuncs.util import get_MEBT_objective_goal_from_BPMoverview
from objFuncs.preset import get_preset, get_tolerance, get_limits
from pyBO import pyBO

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import datetime
from phantasy import caget,fetch_data
import pandas as pd

# user inputs
### check and adjust decision (control knobs) and objectives later

In [ ]:
is_close_to_opt = False     # True for local optimization. 
timespan_for_average = 4.0  # sec  
additional_wait_after_powersupply_ramp  = 0.25 # sec

##== Faraday-Cup PV and goal
FC='REA_BTS40:MTER_N0001:I_RD'
FCgoal = 50e-12

In [ ]:
if is_close_to_opt:
    n_init_budget       = 5   # recommended: number of decision parameters 
    n_global_opt_budget = 0
    n_local_opt_budget  = 25
    n_finetune_budget   = 5   # recommended: less than number of decision parameters 
else:
    n_init_budget       = 25          
    n_global_opt_budget = 25
    n_local_opt_budget  = 25
    n_finetune_budget   = 5    # recommended: less than number of decision parameters 

budget = n_init_budget +n_global_opt_budget +n_local_opt_budget +n_finetune_budget
print(f"budget: {budget}")

# check machineIO, source and beam  

In [ ]:
objFuncs._global_machineIO._test = False
objFuncs._global_machineIO._fetch_data_time_span = timespan_for_average
objFuncs._global_machineIO._ensure_set_timewait_after_ramp = additional_wait_after_powersupply_ramp
objFuncs._global_machineIO._check_chopper_blocking = False

In [ ]:
ion = caget("REA_EXP:ELMT")
Q = int(caget("REA_EXP:Q"))
A = int(caget("REA_EXP:A"))
AQ = A/Q
ion = str(A)+ion+str(Q)
print(ion, 'A/Q=',AQ)

In [ ]:
now0 = datetime.datetime.now()
fname = now0.strftime('%Y%m%d_%H%M')+'['+ion+'][REA][pyBO]FC_CM4_entry'
fname

# preprare decision ( control knob)

In [ ]:
decision_CSETs = ['REA_BTS26:DCH_D1219:I_CSET',
                  'REA_BTS26:DCV_D1219:I_CSET',
                  'REA_BTS30:DCH_D1270:I_CSET',
                  'REA_BTS30:DCV_D1270:I_CSET',
                  ]

In [ ]:
ave, _ = fetch_data(decision_CSETs,0.01)
decision_Lo_limit, decision_Hi_limit = get_limits(decision_CSETs)
decision_limit_half_size = 0.5*(decision_Hi_limit - decision_Lo_limit)
decision_bound_half_size = AQ*0.1*decision_limit_half_size

decision_mid  = np.array([0 if ':DC' in pv else v for v,pv in zip(ave,decision_CSETs)])
decision_tols = 0.01*decision_limit_half_size
decision_min  = decision_mid - decision_bound_half_size
decision_max  = decision_mid + decision_bound_half_size

##== Manually set decision bounds and tolerance
# decision_min = [ -5, -5, -5, -5]
# decision_max = [  5,  5,  5,  5]
# decision_tol = [0.2,0.2,0.2,0.2]
         
assert len(decision_CSETs) == len(decision_min) == len(decision_max) == len(decision_tols)
decition_Lo_limit, decition_Hi_limit = get_limits(decision_CSETs)
decision_min = np.clip(decision_min, a_min = decition_Lo_limit, a_max = None)
decision_max = np.clip(decision_max, a_min = None, a_max = decition_Hi_limit)
assert np.all(np.array(decision_min)<np.array(decision_max))

##== Display decision parameter info
pd.DataFrame(np.array([ave,decision_min,decision_max,decision_tols,decition_Lo_limit,decition_Hi_limit]).T,
             index=decision_CSETs, 
             columns=['current value','decision min','decision max','tol','LoLim','HiLim'])

# preprare objective function

In [ ]:
objective_goal   = {FC: {'more than': FCgoal}}
objective_norm   = {FC: 0.2*FCgoal}
objective_weight = {FC: 1}

##== Display objective info
pd.DataFrame([objective_goal,objective_norm,objective_weight],index=['goal','norm','weight']).T

In [ ]:
obj = objFuncGoals(
    decision_CSETs   = decision_CSETs,
    decision_min     = decision_min,
    decision_max     = decision_max,
    decision_tols    = decision_tols,
    objective_goal   = objective_goal,
    objective_weight = objective_weight,
    objective_norm   = objective_norm,
)

# run Optim

In [ ]:
ctrBO = pyBO.bo_controller(obj,local_optimization = is_close_to_opt)

In [ ]:
ctrBO.init(n_init_budget)
ctrBO.optimize_global(n_global_opt_budget, beta_scheduler='auto')
ctrBO.optimize_local (n_local_opt_budget , beta_scheduler='auto')
ctrBO.fine_tune(n_finetune_budget)
ctrBO.finalize()
for f in ctrBO.plot_callbacks:
    f.close()

### additional optimization

In [ ]:
##== uncomment one (ore more) of the followings to optimize further
# ctrBO.optimize_global(niter=10, beta_scheduler='auto')
# ctrBO.optimize_local (niter=10, beta_scheduler='auto')

##== uncomment the followings too if any of above is uncommented
# ctrBO.fine_tune(niter=2)
# ctrBO.finalize()
# for f in ctrBO.plot_callbacks:
#     f.close()

### plot accumulated best objectives

In [ ]:
fig,ax = plt.subplots(figsize=(4,2),dpi=96)
ctrBO.bo.plot_obj_history(ax=ax, plot_best_only=True)

### set to best solution 

In [ ]:
x_best,y_best_old = ctrBO.bo.best_sofar()
y_best_new = obj(x_best)
print(f'y_best_old: {y_best_old[0]}')
print(f'y_best_new: {y_best_new}')   # check if best solution objective value is consistent
pd.DataFrame([x_best],columns=decision_CSETs,index=['x_best']).T

In [ ]:
obj.save(fname=os.path.join('/projects/rea3/AP-ReA/jupyter-ML/data/log/',fname))

# Visualize Surrogate model

In [ ]:
# plot surrogate mean model of the last epoch. Can take long time for decision dim >= 4
bo = ctrBO.bo
from math import ceil
nplot = int(0.5*len(obj.decision_CSETs))
nrow = ceil(0.5*nplot)
fig,ax = plt.subplots(nrow,2,figsize=(8,3*nrow))
for i in range(nrow):
    for j in range(2):
        n = 2*i+j
        if nrow>1:
            ax_ = ax[i,j]
        else:
            ax_ = ax[j]
        if n >= nplot:
            ax_.set_visible(False)
            break
        bo.plot_model_2D_projection(project_maximum=True,
                                    dim_xaxis = 2*n,
                                    dim_yaxis = 2*n+1,
                                    grid_ponits_each_dim = 16,
                                    fig=fig,ax=ax_);
        ax_.set_xlabel(obj.decision_CSETs[2*n  ])
        ax_.set_ylabel(obj.decision_CSETs[2*n+1])
        ax_.legend()
fig.tight_layout()